# Stochastic Physics Informed Neural Networks (SPINN)

A few key notes:

(1) The code here is tailored to learn hidden physics within stochastic differential equations of the form:

$$ \frac{dx} = g_1(x,u)dt + \sqrt{2g_2(x,u)}dw, $$

where where $x$ is the system state, $u$ is an exogenous input, $w$ is a Gaussian white noise process, $g_1$ is the drift coefficient, and $g_2$ is the diffusion coefficient. Although the drift and diffusion coefficients represent the underlying physics of the stochastic system, these coefficients may not be known or even measurable in practice. As a result, $g_1$, $g_2$, or individual functions that contribute to $g_1$ and $g_2$ can comprise the the hidden physics of the above equation.

(2) This code uses an Euler-Maruyama discretization scheme, which leads to the following description of the state dynamics:

$$ x_{k+1} = x_k + g_1(x_k, u_k)\Delta t + w_k\sqrt{2g_2(x_k,u_k)\Delta t}. $$

Other discretization schemes can be integrated into the code, however.

(3) The code uses unscented transform to propagate stochasticity through the above equations, although other uncertainty propagation methods can be integrated into the code.

(4) Example data is provided for two case studies from the original paper (O'Leary et al., 2021) in the file Data.zip. The first case study is a one-state model for directed colloidal self-assembly with an exogenous input and the second is a two-state competitive Lotka-Volterra model with a coexistence equilibrium. In each case, SPINN learns the drift and diffusion coefficients $g_1$ and $g_2$.

(5)"Examples.zip" contains the results of this exact code run on both examples in (4)

In [ ]:
# Do not write bytecode to maintain clean directories
import sys
sys.dont_write_bytecode = True

# Import required packages and core code
import os
import numpy as np
import core

# Prepare TensorFlow
core.prepare_tensorflow()

In [ ]:
# Load mean and variance data from path chosen by user
# Mean data must be of shape [N, [nx,nu], 1]
# Covariance data must be of shape [N, nx, nx]
# Data is collected at N+1 discrete time points from repeated stochastic system trajectories
# mean_initial/cov_initial consists of mean and covariance estimations at time points 1 to N
# mean_final/cov_final consists of means and covariance estimations at time points 2 to N+1
# The N total time points can come from one or multiple repeated stochastic system trajectories
# In these examples, N = 10^5, which comes from 2000 different 50 time-step trajectories,
# each of which start from different initial conditions
# Each trajectory is repeated is 10^5 times to estimate the mean and covariance

# Colloidal self-assembly system
mean_initial = np.load("CSA/Data/mean_initial_CSA.npy")
mean_final = np.load("CSA/Data/mean_final_CSA.npy")
cov_initial = np.load("CSA/Data/cov_initial_CSA.npy")
cov_final = np.load("CSA/Data/cov_final_CSA.npy")

## Lotka-Volterra system
# mean_initial = np.load("LVE/Data/mean_initial_LVE.npy")
# mean_final = np.load("LVE/Data/mean_final_LVE.npy")
# cov_initial = np.load("LVE/Data/cov_initial_LVE.npy")
# cov_final = np.load("LVE/Data/cov_final_LVE.npy")


In [ ]:
# Enter relevant system information

# Colloidal self-assembly system
nx = 1 # state dimension
nw = 1 # process noise dimension
n = nx + nw # augmented state dimension
nu = 1 # exogenous input dimension
dt = 1 # time discretization

## Lotka-Volterra system
# nx = 2 # state dimension
# nw = 2 # process noise dimension
# n = nx + nw # augmented state dimension
# nu = 0 # exogenous input dimension
# dt = 0.01 # time discretization

In [ ]:
# Get unscented transform (UT) weights
[lam, 
 Wm, 
 Wc] = core.get_weights(nx,
                        nw)

In [ ]:
# Prepare data for training neural network that represents the drift coefficient, D1

# First choose and create path where data will be saved
g1_path = "CSA/Train_g1/"
# g1_path = "LVE/Train_g1/"
os.makedirs(g1_path)

# Now prepare data
[X_train_g1, 
 X_val_g1, 
 X_test_g1, 
 Y_train_g1, 
 Y_val_g1, 
 Y_test_g1, 
 X_mu_g1, 
 X_std_g1, 
 Y_mu_g1, 
 Y_std_g1] = core.g1_train_prep(mean_initial, 
                                mean_final, 
                                cov_initial,  
                                nx, 
                                nu, 
                                nw, 
                                lam,
                                dt,
                                g1_path)

In [ ]:
# Enter ranges of D1 neural network size parameters
n_hidden_layers_g1 = [2, 3] # number of hidden layers
n_hidden_nodes_g1 = [10, 20, 50, 100] # number of hidden nodes

In [ ]:
# Train neural networks that represent D1
core.train_multiple_NNs_g1(X_train_g1, 
                           X_val_g1, 
                           X_test_g1, 
                           Y_train_g1, 
                           Y_val_g1, 
                           Y_test_g1, 
                           n_hidden_layers_g1, 
                           n_hidden_nodes_g1, 
                           nx, 
                           nu, 
                           Wm, 
                           g1_path)

In [ ]:
# Choose g1 neural network that has lowest test MSE
nhl_g1_final = 2 # hidden layers
nhn_g1_final = 10 # hidden nodes
g1_NN_path = os.path.join(g1_path, str(nhl_g1_final) + "_HL_" + str(nhn_g1_final) + "_Nodes/")

g1_NN = core.load_NN(g1_NN_path)

In [ ]:
# Prepare data for training neural network that represents the drift coefficient, D1

# First choose and create path where data will be saved
g2_path = "CSA/Train_g2/"
# g2_path = "LVE/Train_g2/"
os.makedirs(g2_path)

# Now prepare data
[X_train_g2, 
 X_val_g2, 
 X_test_g2, 
 Y_train_g2, 
 Y_val_g2, 
 Y_test_g2, 
 X_mu_g2, 
 X_std_g2, 
 Y_mu_g2, 
 Y_std_g2] = core.g2_train_prep(g1_NN,
                                mean_initial, 
                                mean_final, 
                                cov_initial, 
                                cov_final,
                                X_mu_g1, 
                                X_std_g1, 
                                Y_mu_g1, 
                                Y_std_g1,
                                nx, 
                                nu, 
                                nw, 
                                lam, 
                                Wc,
                                dt, 
                                g2_path)

In [ ]:
# Enter ranges of g2 neural network size parameters
n_hidden_layers_g2 = [2, 3] # number of hidden layers
n_hidden_nodes_g2 = [10, 20, 50, 100] # number of hidden nodes

In [ ]:
# Train neural networks that represent g2
core.train_multiple_NNs_g2(X_train_g2, 
                           X_val_g2, 
                           X_test_g2, 
                           Y_train_g2, 
                           Y_val_g2,
                           Y_test_g2, 
                           n_hidden_layers_g2, 
                           n_hidden_nodes_g2, 
                           nx, 
                           nu,
                           g2_path)

In [ ]:
# Choose g2 neural network that has lowest test MSE
nhl_g2_final = 2 # hidden layers
nhn_g2_final = 20 # hidden nodes
g2_NN_path = os.path.join(g2_path, str(nhl_g2_final) + "_HL_" + str(nhn_g2_final) + "_Nodes/")

g2_NN = core.load_NN(g2_NN_path)

In [ ]:
# Plot learned hidden physics and true hidden physics

# Plot g1

# Colloidal self-assembly system
core.plot_reconstruction_CSA(g1_NN, X_mu_g1, X_std_g1, Y_mu_g1, Y_std_g1, "g1", g1_path)

# Lotka-Volterra system
# core.plot_reconstruction_LVE(g1_NN, X_mu_g1, X_std_g1, Y_mu_g1, Y_std_g1, "g1", g1_path)

In [ ]:
# Plot g2

# Colloidal self-assembly system
core.plot_reconstruction_CSA(g2_NN, X_mu_g2, X_std_g2, Y_mu_g2, Y_std_g2, "g2", g2_path)

# Lotka-Volterra system
# core.plot_reconstruction_LVE(g2_NN, X_mu_g2, X_std_g2, Y_mu_g2, Y_std_g2, "g2", g2_path)